# Weekly COVID-19 Vaccine Dashboard Update
Center for Human Dynamics in the Mobile Age (HDMA) at San Diego State University <br>

Jessica Embury

### IMPORTS

In [33]:
#import modules
from arcgis.gis import GIS
from arcgis import geometry
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.features import FeatureLayerCollection
from arcgis.features import FeatureLayer

import numpy as np
import pandas as pd
import webbrowser

#ignore pandas slice warning
pd.options.mode.chained_assignment = None  # default='warn'

### DATE, FILE PATHS, & AGOL IDS

In [34]:
###########################
###SET DATA THROUGH DATE###
###########################
first_date_in_range = '07/07/21' # Change to 1st day in 7-day range using 'MM/DD/YY' format
most_recent_date = '07/13/2021'  # Change to data through date using 'MM/DD/YYYY' format
max_cases = 14117  # number of max covid cases in any sd zip code

##########################
###SET PATHS IN AND OUT###
##########################
#PATHS IN
#additional needed zip info - community name and population
zips_info_path = './data/zip_coords_pop.csv'

#paths out
path_out = './data/sd_zip_vaccines.csv' #feature overwrite
path_out2 = './data/sd_zip_vaccines_alldates.csv' # date columns
path_out3 = '../covid_data/vaccines/sd_zip_vaccines_{}.csv'.format(most_recent_date.replace('/', '')) # overwrite data archive
path_out4 = './data/sd_zip_vaccines_7d.csv' #feature overwrite for 7-day layer
path_db = 'C:/Users/jesse/Dropbox/Mapping-Vulearable-Pop-Tasks/SD-County-Data/COVID-Data-Share-at-HDMA-Center/Vaccination/SD_Zip_COVID_Vaccine_{}.csv'.format(most_recent_date.replace('/', '')) # dropbox

vacc_count_map_id = '3a5a90a0b43b42bb823ab4136b8d58e9'
conf_cases_map_id = '9b08e97e950f4cd79743e3a8217d4508'
vacc_dash = 'https://arcg.is/15ePOz'

vacc_7d_layer_id = 'dc90f581b4914e52b382b086423fb6b7'
vacc_7d_map_id = '19e5ea7a388c4eb18c70392b7d8f6a03'
vacc_7d_dash = 'https://arcg.is/b4iHr'

vacc_hpi_id = '694f95f80e6849608470e5ecfc1448d2'
vacc_hpi_dash = 'https://experience.arcgis.com/experience/16370187fbb64720b468483638c02e78/'

vacc_hpi_7d_id = '7ed91cc1f7454df6b750c784b7ce70d3'
vacc_hpi_7d_dash = 'https://experience.arcgis.com/experience/1cc7b911604e4a1ab3bd0fbe95227b35/'

### MANUAL COPY PDF TO CSV, THEN FORMAT

In [35]:
# save info from cumulative vacc by zip PDF to csv file. Copy/paste data will result in all data in one column.
# separate data from raw unformatted csv
raw = pd.read_csv('./data/zip_vacc_raw.csv')

# create empty DF
cols = ['Zip Code', 'Vacc Count', 'Rate Per 1000']
t1 = pd.DataFrame(columns = cols)

# zip code
raw1 = raw.iloc[0::3, :].reset_index()
print(len(raw1))
# vacc count
raw2 = raw.iloc[1::3, :].reset_index()
print(len(raw2))
# rate
raw3 = raw.iloc[2::3, :].reset_index()
print(len(raw3))

# add raw data to t1 data frame
for i in range(113):
    temp = [raw1['vacc'][i], raw2['vacc'][i], raw3['vacc'][i]]
    t1 = t1.append({'Zip Code': temp[0], 'Vacc Count': temp[1], 'Rate Per 1000': temp[2]}, ignore_index=True)

print(len(t1))
t1.head()

113
113
113
113


,Zip Code,Vacc Count,Rate Per 1000
0,91901,"8,293",537
1,92069,"26,894",640.8
2,91902,"12,265",826.4
3,92070,543,**
4,91905,634,373


In [36]:
#format numbers in counts column
t1 = t1.fillna(0)

for i, row in t1.iterrows():
    try:
        t1['Vacc Count'][i] = str(t1['Vacc Count'][i].replace(',', '')).replace('***','0')
        t1['Vacc Count'][i] = int(t1['Vacc Count'][i])
    except:
        print(t1['Zip Code'][i], t1['Vacc Count'][i])
    
    try:
        t1['Rate Per 1000'][i] = str(t1['Rate Per 1000'][i].replace('***', '0').replace('**', '0').replace(',','')).replace('*','')
        t1['Rate Per 1000'][i] = float(t1['Rate Per 1000'][i])
    except:
        print(t1['Zip Code'][i], t1['Rate Per 1000'][i])

t1['Vacc Count'] = t1['Vacc Count'].astype(int)
print(type(t1['Vacc Count'][0]))

t1['Rate Per 1000'] = t1['Rate Per 1000'].astype(float)
print(type(t1['Rate Per 1000'][0]))

t1.head()

<class 'numpy.int32'>
<class 'numpy.float64'>


,Zip Code,Vacc Count,Rate Per 1000
0,91901,8293,537.0
1,92069,26894,640.8
2,91902,12265,826.4
3,92070,543,0.0
4,91905,634,373.0


In [37]:
# supplemental zip code data
zips = pd.read_csv(zips_info_path)
zips.head()

# merge vaccine info with zip code info
t1['Zip Code'] = t1['Zip Code'].astype(str)
zips['Zip'] = zips['Zip'].astype(str)

df = pd.merge(zips, t1, how = 'right', left_on = 'Zip', right_on = 'Zip Code')
del df['Zip Code']

# add data date
df['Date'] = most_recent_date
df

,Zip,Community,2018_population,Latitude,Longitude,Vacc Count,Rate Per 1000,Date
0,91901,Alpine,17885,32.805710,-116.695537,8293,537.0,07/13/2021
1,92069,San Marcos,49734,33.178965,-117.158328,26894,640.8,07/13/2021
2,91902,Bonita,17375,32.671583,-117.015068,12265,826.4,07/13/2021
3,92070,Santa Ysabel,879,33.188952,-116.715840,543,0.0,07/13/2021
4,91905,Boulevard,2014,32.718365,-116.305469,634,373.0,07/13/2021
...,...,...,...,...,...,...,...,...
108,92065,Ramona,35628,33.062768,-116.843980,16377,529.9,07/13/2021
109,92536,Aguanga,491,33.398676,-116.720547,237,0.0,07/13/2021
110,92066,Ranchita,342,33.193275,-116.490088,134,0.0,07/13/2021
111,92672,San Clemente,4529,33.389143,-117.571496,1259,353.6,07/13/2021


In [38]:
# prep data for dated column csv
df2 = pd.read_csv(path_out2)
print(len(df2))
df2['Zip'] = df2['Zip'].astype(str)
df2 = df2.merge(df[['Zip', 'Vacc Count']], how = "right", on = "Zip")
print(len(df2))
df2 = df2.rename(columns = {'Vacc Count': most_recent_date})
df2.head()

113
113


,Zip,Community,Latitude,Longitude,2/23/2021,2/24/2021,2/25/2021,2/26/2021,2/27/2021,2/28/2021,3/1/2021,3/2/2021,3/3/2021,3/4/2021,3/5/2021,3/6/2021,3/7/2021,3/8/2021,3/9/2021,3/10/2021,3/11/2021,3/12/2021,3/13/2021,3/14/2021,3/15/2021,3/16/2021,3/17/2021,3/18/2021,3/19/2021,3/20/2021,3/21/2021,3/22/2021,3/23/2021,3/24/2021,3/25/2021,3/26/2021,3/27/2021,3/28/2021,3/29/2021,3/30/2021* (No update),...,5/23/2021,5/24/2021,5/25/2021,5/26/2021,5/27/2021,5/28/2021,5/29/2021,5/30/2021* (No update),5/31/2021,6/1/2021,6/2/2021,6/3/2021,6/4/2021,6/5/2021,6/6/2021,6/7/2021,6/8/2021,6/9/2021,6/10/2021,6/11/2021,6/12/2021,6/13/2021,6/14/2021,6/15/2021,6/16/2021* (No update),06/17/2021,06/18/2021,06/19/2021,06/20/2021,06/21/2021,06/22/2021,06/23/2021,06/24/2021,06/25/2021,06/26/2021,06/27/2021,06/28/2021,06/29/2021,07/06/2021,07/13/2021
0,91901,Alpine,32.805710,-116.695537,3095.0,3167.0,3210.0,3238.0,3263.0,3290.0,3327.0,3358.0,3440,3470,3517,3526,3547,3571,3650,3725,3771,3831,3903,3925,3963,4015,4081,4146,4200,4259,4305,4337,4390,4429,4478,4517,4566,4589,4620,4620,...,7343,7352,7393,7412,7450,7473,7494,7494,7518,7529,7598,7712,7730,7746,7817,7830,7849,7879,7893,7903,7917,7923,7935,7952,7952,7988,7999,8009,8022,8025,8044,8054,8069,8080,8100,8108,8113,8140,8211,8293
1,92069,San Marcos,33.178965,-117.158328,5478.0,5582.0,5686.0,5764.0,5897.0,6049.0,6212.0,6380.0,6554,6689,6817,6918,7065,7192,7329,7437,7658,7757,7947,7984,8060,8172,8310,8508,8659,8889,9031,9138,9332,9496,9698,9877,10068,10171,10316,10316,...,21529,21563,21718,21858,21961,22058,22165,22165,22271,22303,23130,24196,24286,24377,24585,24660,24782,24975,25083,25140,25194,25232,25334,25429,25429,25555,25608,25683,25717,25745,25825,25894,25970,26043,26111,26143,26174,26231,26505,26894
2,91902,Bonita,32.671583,-117.015068,4463.0,4531.0,4580.0,4650.0,4712.0,4782.0,4876.0,4968.0,5065,5129,5206,5221,5263,5330,5448,5560,5614,5676,5789,5835,5888,5954,6067,6165,6241,6346,6416,6466,6561,6637,6704,6783,6843,6888,6931,6931,...,10860,10888,10911,10971,10996,11071,11105,11105,11137,11149,11278,11437,11459,11486,11593,11608,11637,11663,11687,11699,11719,11729,11742,11784,11784,11818,11846,11864,11879,11894,11912,11941,11973,12001,12017,12029,12037,12054,12141,12265
3,92070,Santa Ysabel,33.188952,-116.715840,210.0,213.0,217.0,219.0,219.0,224.0,229.0,231.0,234,231,232,232,234,236,240,243,242,244,250,247,252,258,261,261,267,271,274,280,287,290,292,301,303,304,310,310,...,474,474,475,475,476,478,482,482,484,484,492,507,508,509,511,511,513,516,516,517,517,518,521,523,523,529,530,530,531,531,532,532,535,535,536,537,537,538,540,543
4,91905,Boulevard,32.718365,-116.305469,221.0,225.0,226.0,228.0,228.0,229.0,230.0,234.0,236,238,246,247,247,244,250,252,250,252,269,269,271,293,297,304,311,311,312,314,317,333,334,336,337,339,341,341,...,530,536,536,539,540,540,542,542,546,546,563,580,585,587,588,588,594,596,598,600,600,600,604,606,606,608,610,610,610,610,612,613,615,615,619,619,619,623,627,634


In [39]:
# calculate 7-day changes
df3 = df2[['Zip', df2.columns[-2], df2.columns[-1]]]
df3 = df3.fillna(0)
df3['7-Day Vacc Count'] = df3[df3.columns[2]].astype(int) - df3[df3.columns[1]].astype(int)

# add 7-day date range column
df3['7-Day Date Range'] = '{} - {}'.format(first_date_in_range , df2.columns[-1].replace('2021', '21'))

del df3[df3.columns[1]]
del df3[df3.columns[1]]

print(len(df3))
df3.head()

113


,Zip,7-Day Vacc Count,7-Day Date Range
0,91901,82,07/07/21 - 07/13/21
1,92069,389,07/07/21 - 07/13/21
2,91902,124,07/07/21 - 07/13/21
3,92070,3,07/07/21 - 07/13/21
4,91905,7,07/07/21 - 07/13/21


In [40]:
# merge 7-day vaccines to main df
df = df.merge(df3, how = 'left', on = 'Zip')
df.head()

# calculate 7 day rates
df['7-Day Rate Per 1000'] = 0.0

#7d rate per 1000
for i, row in df.iterrows():
    if df['2018_population'][i] >= 10000:
        df['7-Day Rate Per 1000'][i] = round(df['7-Day Vacc Count'][i]/df['2018_population'][i]*1000, 1)

print(len(df))
df.tail()

113


,Zip,Community,2018_population,Latitude,Longitude,Vacc Count,Rate Per 1000,Date,7-Day Vacc Count,7-Day Date Range,7-Day Rate Per 1000
108,92065,Ramona,35628,33.062768,-116.843980,16377,529.9,07/13/2021,162,07/07/21 - 07/13/21,4.5
109,92536,Aguanga,491,33.398676,-116.720547,237,0.0,07/13/2021,4,07/07/21 - 07/13/21,0.0
110,92066,Ranchita,342,33.193275,-116.490088,134,0.0,07/13/2021,-1,07/07/21 - 07/13/21,0.0
111,92672,San Clemente,4529,33.389143,-117.571496,1259,353.6,07/13/2021,8,07/07/21 - 07/13/21,0.0
112,92067,Rancho Santa Fe,9895,33.018435,-117.224304,7794,906.0,07/13/2021,51,07/07/21 - 07/13/21,0.0


In [41]:
# save feature overwrite info
df.to_csv(path_out, index=False)
df.to_csv(path_out3, index=False)
df.to_csv(path_out4, index=False)

# save dated columns to csv
df2.to_csv(path_out2, index=False)
df2.to_csv(path_db, index=False)

In [42]:
# get max count and rate for map updates
max_count = df['Vacc Count'].max()
max_rate = df['Rate Per 1000'].max()
max_7d = df['7-Day Vacc Count'].max()
max_7d_rate = df['7-Day Rate Per 1000'].max()
print(max_count, max_rate, max_7d, max_7d_rate)

62150 999.0 897 25.5


### CONNECT TO AGOL ACCOUNT

In [43]:
gis = GIS("pro")

### FUNCTIONS

In [44]:
def get_map (map_id):
    '''
    GET MAP DATA FOR SYMBOLOGY CHANGES
    '''
    m = gis.content.get(map_id)
    data = m.get_data()
    #Include the below line for prettified JSON
    #print(json.dumps(data, indent=4, sort_keys=True))
    print(m)
    
    return data
    
def update_map (map_id, data):
    '''
    UPDATE MAP TO SAVE CHANGES
    '''
    m = gis.content.get(map_id)
    # Set the item_properties to include the desired update
    properties = {"text": json.dumps(data)}
    # 'Commit' the updates to the Item
    update = m.update(item_properties=properties)
    
    return update

### FEATURE LAYER OVERWRITE

In [45]:
#get feature layer containing updated data for maps associated with the 7-day COVID-19 vaccines
layer = gis.content.get(vacc_7d_layer_id)
layer

layer_collection = FeatureLayerCollection.fromitem(layer)

#call the overwrite() method which can be accessed using the manager property
layer_collection.manager.overwrite(path_out4)

{'success': True}

### UPDATE MAP SYMBOLOGY

In [46]:
# TOTAL VACC MAP
#get map data
vacc1_data = get_map(vacc_count_map_id)

#adjust symbology for graduated points to reflect new max/min
#MAX VACC COUNT
vacc1_data['operationalLayers'][4]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_count.item()
vacc1_data['operationalLayers'][4]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_count.item()

vacc1_data['operationalLayers'][5]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_count.item()
vacc1_data['operationalLayers'][5]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_count.item()

#update map to save changes
vacc1_update = update_map(vacc_count_map_id, vacc1_data)
vacc1_update

<Item title:"SD COVID-19 Vaccination Counts" type:Web Map owner:jembury_SDSUGeo>


True

In [47]:
# TOTAL COVID MAP
#get map data
vacc2_data = get_map(conf_cases_map_id)

#adjust symbology for graduated points to reflect new max/min
#MAX VACC COUNT
vacc2_data['operationalLayers'][4]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_cases
vacc2_data['operationalLayers'][4]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_cases

vacc2_data['operationalLayers'][5]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_cases
vacc2_data['operationalLayers'][5]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_cases

#update map to save changes
vacc2_update = update_map(conf_cases_map_id, vacc2_data)
vacc2_update

<Item title:"COVID-19 Confirmed Cases - vacc dash" type:Web Map owner:jembury_SDSUGeo>


True

In [48]:
# 7 DAY VACC MAP
#get map data
vacc3_data = get_map(vacc_7d_map_id)

#adjust symbology for graduated points to reflect new max/min
#MAX VACC COUNT
vacc3_data['operationalLayers'][4]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_7d.item()
vacc3_data['operationalLayers'][4]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_7d.item()

vacc3_data['operationalLayers'][5]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_7d.item()
vacc3_data['operationalLayers'][5]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_7d.item()

#update map to save changes
vacc3_update = update_map(vacc_7d_map_id, vacc3_data)
vacc3_update

<Item title:"SD COVID-19 Vaccination Counts (7-Day)" type:Web Map owner:jembury_SDSUGeo>


True

In [49]:
# TOTAL VACCINE & HPI Dashboard
#get map data
vacc4_data = get_map(vacc_hpi_id)

#adjust symbology for graduated points to reflect new max/min
#MAX VACC COUNT
# [7], [8], [9] --> size only, no color ramp
vacc4_data['operationalLayers'][9]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_count.item()
vacc4_data['operationalLayers'][9]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_count.item()

vacc4_data['operationalLayers'][10]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_count.item()
vacc4_data['operationalLayers'][10]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_count.item()

vacc4_data['operationalLayers'][11]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_count.item()
vacc4_data['operationalLayers'][11]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_count.item()

#size info
vacc4_data['operationalLayers'][12]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_count.item()
vacc4_data['operationalLayers'][12]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][1]['maxDataValue'] = max_count.item()
# color info
vacc4_data['operationalLayers'][12]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][1]['maxSliderValue'] = max_rate.item()
vacc4_data['operationalLayers'][12]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_rate.item()

#size info
vacc4_data['operationalLayers'][13]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_count.item()
vacc4_data['operationalLayers'][13]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][1]['maxDataValue'] = max_count.item()
# color info
vacc4_data['operationalLayers'][13]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][1]['maxSliderValue'] = max_rate.item()
vacc4_data['operationalLayers'][13]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_rate.item()

#size info
vacc4_data['operationalLayers'][14]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_count.item()
vacc4_data['operationalLayers'][14]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_count.item()
# color info
vacc4_data['operationalLayers'][14]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][1]['maxSliderValue'] = max_rate.item()
vacc4_data['operationalLayers'][14]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][1]['maxDataValue'] = max_rate.item()

#update map to save changes
vacc4_update = update_map(vacc_hpi_id, vacc4_data)
vacc4_update

<Item title:"COVID-19 Vaccines (Total) and Healthy Places Index (HPI)" type:Web Map owner:jembury_SDSUGeo>


True

In [50]:
# 7-DAY VACCINE & HPI Dashboard
#get map data
vacc5_data = get_map(vacc_hpi_7d_id)

#adjust symbology for graduated points to reflect new max/min
#MAX VACC COUNT
# [7], [8], [9] --> size only, no color ramp
vacc5_data['operationalLayers'][9]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_7d.item()
vacc5_data['operationalLayers'][9]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_7d.item()

vacc5_data['operationalLayers'][10]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_7d.item()
vacc5_data['operationalLayers'][10]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_7d.item()

vacc5_data['operationalLayers'][11]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_7d.item()
vacc5_data['operationalLayers'][11]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_7d.item()

#size info
vacc5_data['operationalLayers'][12]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_7d.item()
vacc5_data['operationalLayers'][12]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][1]['maxDataValue'] = max_7d.item()
# color info
vacc5_data['operationalLayers'][12]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][1]['maxSliderValue'] = max_7d_rate.item()
vacc5_data['operationalLayers'][12]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_7d_rate.item()

#size info
vacc5_data['operationalLayers'][13]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_7d.item()
vacc5_data['operationalLayers'][13]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][1]['maxDataValue'] = max_7d.item()
# color info
vacc5_data['operationalLayers'][13]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][1]['maxSliderValue'] = max_7d_rate.item()
vacc5_data['operationalLayers'][13]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_7d_rate.item()

#size info
vacc5_data['operationalLayers'][14]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_7d.item()
vacc5_data['operationalLayers'][14]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][1]['maxDataValue'] = max_7d.item()
# color info
vacc5_data['operationalLayers'][14]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][1]['maxSliderValue'] = max_7d_rate.item()
vacc5_data['operationalLayers'][14]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_7d_rate.item()

#update map to save changes
vacc5_update = update_map(vacc_hpi_7d_id, vacc5_data)
vacc5_update

<Item title:"COVID-19 Vaccines (7-Day) and Healthy Places Index (HPI)" type:Web Map owner:jembury_SDSUGeo>


True

### VIEW UPDATED DASHBOARD

In [51]:
webbrowser.open(vacc_dash, new=2)
webbrowser.open(vacc_7d_dash, new=2)
webbrowser.open(vacc_hpi_dash, new=2)
webbrowser.open(vacc_hpi_7d_dash, new=2)

True